<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/transformers/finetuning_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://huggingface.co/transformers/examples.html#roberta-bert-and-masked-language-modeling
# https://www.kaggle.com/jihunlorenzopark/pretrain-with-another-twitter-dataset

In [16]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sat May  2 13:11:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------

In [17]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (361/361), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 26096 (delta 178), reused 322 (delta 163), pack-reused 25735
Receiving objects: 100% (26096/26096), 15.38 MiB | 14.30 MiB/s, done.
Resolving deltas: 100% (18216/18216), done.


In [18]:
import os
os.chdir("transformers")
!pip install .

Processing /content/transformers/transformers
  Created wheel for transformers: filename=transformers-2.8.0-cp36-none-any.whl size=595726 sha256=6cec2d43309d3cc8aa55853534fe0753375671288608500e64df08b8d72f81bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-xas3cq0l/wheels/24/4e/ad/9bac5e50177cae64c9b8ec988ae7d7d1f114f296276d2080f5
Successfully built transformers
  Found existing installation: transformers 2.8.0
    Uninstalling transformers-2.8.0:
      Successfully uninstalled transformers-2.8.0


In [0]:
# !wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
# !unzip wikitext-2-raw-v1.zip

In [0]:
# !python examples/run_language_modeling.py \
#     --output_dir=output \
#     --model_type=roberta \
#     --model_name_or_path=roberta-base \
#     --do_train \
#     --train_data_file=wikitext-2-raw/wiki.test.raw \
#     --do_eval \
#     --eval_data_file=wikitext-2-raw/wiki.test.raw \
#     --mlm

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd, re, numpy as np


In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
ROBERTA_PATH = INPUT_PATH + 'tf-roberta/'
EMOTION_PATH = INPUT_PATH + 'training-data/emotion/text_emotion.csv'
SENTIMENT140_PATH = INPUT_PATH + 'training-data/sentiment140/training.1600000.processed.noemoticon.csv'
TWITTER_SENTIMENTS_PATH = INPUT_PATH + 'training-data/twitter-sentiment/Sentiment Analysis Dataset 2.csv'

In [0]:
def remove_html(text):
    text = re.sub("&quot;", '"', text)
    text = re.sub("&gt;", ">", text)
    text = re.sub("&lt;", "<", text)
    text = re.sub("&le;", "≤", text)
    text = re.sub("&ge;", "≥", text)
    text = re.sub("&amp;", "&", text)
    return text

def remove_twitter_id(text):
    text = text.strip()
    text = " ".join([i for i in text.split(" ") if "@" not in i])
    text = text.strip()
    return text

def to_text(df, filename):
    f = open(filename, 'w')
    for c in df["content"].values:
        f.write(" " + c.strip() + "\n\n")
    f.close()

def pandas_to_train_test_file(df, filename):
    df_train, df_test = train_test_split(df, test_size=0.1, random_state=333)
    to_text(df_train, "../%s.train.txt"%filename)
    to_text(df_test, "../%s.test.txt"%filename)

In [24]:
emotion_df = pd.read_csv(EMOTION_PATH)
emotion_df['content'] = emotion_df['content'].astype(str).str.lower()
emotion_df["content"] = emotion_df["content"].apply(remove_html)
emotion_df["content"] = emotion_df["content"].apply(remove_twitter_id)
emotion_df = emotion_df[["content"]]
print(emotion_df.shape[0])
emotion_df.head()

40000


,content
0,i know i was listenin to bad habit earlier an...
1,layin n bed with a headache ughhhh...waitin o...
2,funeral ceremony...gloomy friday...
3,wants to hang out with friends soon!
4,we want to trade with someone who has houston ...


In [25]:
sentiment140_df = pd.read_csv(SENTIMENT140_PATH, encoding = "ISO-8859-1", engine='python', header=None)
sentiment140_df.columns = [0, 1, 2, 3, 4, "content"]
sentiment140_df['content'] = sentiment140_df['content'].astype(str).str.lower()
sentiment140_df["content"] = sentiment140_df["content"].apply(remove_html)
sentiment140_df["content"] = sentiment140_df["content"].apply(remove_twitter_id)
sentiment140_df = sentiment140_df[["content"]]
print(sentiment140_df.shape[0])
sentiment140_df.head()

1600000


,content
0,"http://twitpic.com/2y1zl - awww, that's a bumm..."
1,is upset that he can't update his facebook by ...
2,i dived many times for the ball. managed to sa...
3,my whole body feels itchy and like its on fire
4,"no, it's not behaving at all. i'm mad. why am ..."


In [26]:
twitter_df = pd.read_csv(TWITTER_SENTIMENTS_PATH, error_bad_lines=False)
twitter_df.columns = [0, 1, 2, "content"]
twitter_df['content'] = twitter_df['content'].astype(str).str.lower()
twitter_df["content"] = twitter_df["content"].apply(remove_html)
twitter_df["content"] = twitter_df["content"].apply(remove_twitter_id)
twitter_df = twitter_df[["content"]]
print(twitter_df.shape[0])
twitter_df.head()

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


1578612


,content
0,is so sad for my apl friend.............
1,i missed the new moon trailer...
2,omg its already 7:30 :o
3,.. omgaga. im sooo im gunna cry. i've been at...
4,i think mi bf is cheating on me!!! t_t


In [27]:
full_df = pd.concat([emotion_df, sentiment140_df, twitter_df])
print(full_df.shape)
pandas_to_train_test_file(full_df, "twitter_3m")
full_df.head()

(3218612, 1)


,content
0,i know i was listenin to bad habit earlier an...
1,layin n bed with a headache ughhhh...waitin o...
2,funeral ceremony...gloomy friday...
3,wants to hang out with friends soon!
4,we want to trade with someone who has houston ...


In [0]:
# !python examples/run_language_modeling.py \
#     --output_dir=../roberta-tmp \
#     --model_type=roberta \
#     --model_name_or_path=roberta-base \
#     --do_train \
#     --train_data_file=../emotion.train.txt \
#     --do_eval \
#     --eval_data_file=../emotion.test.txt \
#     --mlm

In [29]:
!python examples/run_language_modeling.py \
    --output_dir='/content/drive/My Drive/input/models/roberta-twitter3m' \
    --model_type=roberta \
    --model_name_or_path=roberta-base \
    --do_train \
    --train_data_file=../twitter_3m.train.txt \
    --do_eval \
    --eval_data_file=../twitter_3m.test.txt \
    --save_steps=0 \
    --mlm

Streaming output truncated to the last 5000 lines.
Iteration:  63% 8364/13319 [1:25:52<50:42,  1.63it/s]
Iteration:  63% 8365/13319 [1:25:53<50:53,  1.62it/s]
Iteration:  63% 8366/13319 [1:25:53<50:57,  1.62it/s]
Iteration:  63% 8367/13319 [1:25:54<50:58,  1.62it/s]
Iteration:  63% 8368/13319 [1:25:54<50:50,  1.62it/s]
Iteration:  63% 8369/13319 [1:25:55<50:44,  1.63it/s]
Iteration:  63% 8370/13319 [1:25:56<50:39,  1.63it/s]
Iteration:  63% 8371/13319 [1:25:56<50:47,  1.62it/s]
Iteration:  63% 8372/13319 [1:25:57<50:42,  1.63it/s]
Iteration:  63% 8373/13319 [1:25:58<50:37,  1.63it/s]
Iteration:  63% 8374/13319 [1:25:58<50:36,  1.63it/s]
Iteration:  63% 8375/13319 [1:25:59<50:33,  1.63it/s]
Iteration:  63% 8376/13319 [1:25:59<50:47,  1.62it/s]
Iteration:  63% 8377/13319 [1:26:00<50:39,  1.63it/s]
Iteration:  63% 8378/13319 [1:26:01<50:34,  1.63it/s]
Iteration:  63% 8379/13319 [1:26:01<50:32,  1.63it/s]
Iteration:  63% 8380/13319 [1:26:02<50:31,  1.63it/s]
Iteration:  63% 8381/13319 [1:2